In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
from collections import defaultdict

In [2]:
dat = pd.read_csv('train.csv', index_col='row_id')

the test data is on a Monday afternoon, produce two test sets in Monday after for checking

In [3]:
test_periods = [
    ['1991-08-26 12:00:00', '1991-09-02 24:00:00'],
    ['1991-09-02 12:00:00', '1991-09-02 24:00:00'],
    ['1991-09-09 12:00:00', '1991-09-09 24:00:00'],
    ['1991-09-16 12:00:00', '1991-09-16 24:00:00']]

In [7]:
def preprocess(dat):
    
    time_mapper = {}
    ii = 0
    for h in range(24):
        for mm in ['00','20','40']:
            hh = '{0:02d}'.format(h)
            time_mapper[hh+':'+mm] = ii
            ii += 1
    
    # define the road/day/time_stamp
    dat['unique'] = dat['x'].astype(str) + dat['y'].astype(str) + dat['direction']
    dat['day'] = pd.to_datetime(dat['time']).dt.weekday
    dat['time_stamp'] = dat['time'].apply(lambda x:time_mapper[x.split()[1][:5]])
    uniques = dat['unique'].unique()

    # define the medians
    tmp = dat.groupby(['unique','day','time_stamp']).agg({'congestion':np.median})
    median_mapper = tmp.to_dict()['congestion']
    
    # define the modes
    tmp = dat.groupby(['unique','day','time_stamp']).\
                    agg({'congestion': lambda x: stats.mode(x).mode[0]})
    mode_mapper = tmp.to_dict()['congestion']
    
    # define the diff
    diff_mapper = {}
    for unique in uniques:
        tmp = dat.loc[dat['unique']==unique,['day','time_stamp','congestion']]
        tmp['diff'] = tmp['congestion'].diff()
        tmp = tmp.groupby(['day','time_stamp']).agg({'diff':lambda x: sum(x==0)/len(x)})
        for idx in tmp.index:
            diff_mapper[(unique, idx[0], idx[1])] = tmp.at[idx,'diff']
    
    # define the total
    sums = dat.groupby('time').agg({'congestion':sum})
    
    congestions = dat[['unique','congestion','time']].set_index('time')
    congestions = congestions.groupby(['unique','time']).mean().unstack('unique')
    congestions.columns = congestions.columns.droplevel(0)
    
    cols = congestions.columns.to_list()
    for col in cols:
        col_new = col + '-1'
        congestions[col_new] = np.hstack((0, congestions[col][:-1]))
        col_new = col + '-2'
        congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
    congestions = congestions.copy()
    congestions = congestions.drop(index=congestions.index[:2], axis=1)
    
    sums.columns = ['total']
    sums['total-1'] = np.hstack((0, sums['total'][:-1]))
    sums['total-2'] = np.hstack((0, 0, sums['total'][:-2]))
    sums = sums.drop(index=sums.index[:2], axis=1)
    
    congestions['time_stamp'] = \
       pd.Series(congestions.index).apply(lambda x:time_mapper[x.split()[1][:5]]).values
    congestions['day'] = \
       pd.to_datetime(pd.Series(congestions.index)).dt.weekday.values
    
    congestions = pd.merge(congestions, sums, left_on='time', right_on='time', how='outer')
    
    return congestions, uniques, median_mapper, time_mapper, diff_mapper, mode_mapper

In [8]:
congestions, uniques, median_mapper, time_mapper, diff_mapper, mode_mapper = preprocess(dat)

/tmp/ipykernel_63786/2627335551.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
/tmp/ipykernel_63786/2627335551.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, congestions[col][:-1]))


In [9]:
congestions

,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day,total,total-1,total-2
time,,,,,,,,,,,,,,,,,,,,,
1991-04-01 00:40:00,70.0,49.0,24.0,31.0,51.0,57.0,91.0,42.0,41.0,35.0,...,70.0,29.0,29.0,57.0,26.0,2,0,2735,2730,2691
1991-04-01 01:00:00,70.0,49.0,24.0,36.0,51.0,46.0,49.0,39.0,44.0,58.0,...,80.0,29.0,29.0,48.0,57.0,3,0,2692,2735,2730
1991-04-01 01:20:00,70.0,49.0,24.0,29.0,53.0,83.0,62.0,36.0,46.0,46.0,...,79.0,29.0,29.0,44.0,48.0,4,0,2658,2692,2735
1991-04-01 01:40:00,70.0,49.0,24.0,15.0,48.0,38.0,39.0,36.0,51.0,43.0,...,77.0,29.0,29.0,44.0,44.0,5,0,2668,2658,2692
1991-04-01 02:00:00,70.0,49.0,65.0,20.0,27.0,46.0,39.0,35.0,56.0,49.0,...,79.0,29.0,29.0,44.0,44.0,6,0,2904,2668,2658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-09-30 10:20:00,43.0,29.0,60.0,23.0,63.0,61.0,42.0,47.0,51.0,56.0,...,66.0,24.0,35.0,33.0,41.0,31,0,3058,2977,3008
1991-09-30 10:40:00,47.0,28.0,35.0,24.0,62.0,63.0,32.0,44.0,57.0,65.0,...,69.0,22.0,24.0,49.0,33.0,32,0,2922,3058,2977
1991-09-30 11:00:00,40.0,31.0,53.0,22.0,65.0,58.0,38.0,44.0,60.0,62.0,...,68.0,20.0,22.0,47.0,49.0,33,0,2952,2922,3058


In [11]:
def calconnect():
    connected = defaultdict(list)
    tmp ={'E':['0','1','2'],
          'N':['0','1','2','3'],
          'W':['0','1','2'],
          'S':['0','1','2','3']}
    diag = []
    for unique in uniques:
        if unique[3] != 'B': diag.append(unique)
            
    for unique in uniques:
        if unique[3] != 'B':  connected[unique] = diag[:]
        if unique[2] == 'E' or unique[2] == 'W':
            y = unique[1]
            for x in tmp[unique[2]]:
                new = x + y + 'EB'
                if new in uniques: connected[unique].append(new)
                new = x + y + 'WB'
                if new in uniques: connected[unique].append(new)
        else:
            x = unique[0]
            for y in tmp[unique[2]]:
                new = x + y + 'NB'
                if new in uniques: connected[unique].append(new)
                new = x + y + 'SB'
                if new in uniques: connected[unique].append(new)
    
    for key,item in connected.items():
        for idx in range(len(item)):
            item[idx] += '-1'
            
    return connected

In [69]:
connected = calconnect()

In [30]:
def assemble(col):
    cols = connected[col][:]
    #cols = [col for col in congestions.columns.to_list() if '-' in col]
    cols.append('time_stamp')
    cols.append('day')
    cols.append('total-1')
    dat = congestions[cols].copy()  #the copy is needed
    dat['median'] = dat.apply(\
                        lambda x: median_mapper[(col, x['day'].astype(int), 
                                                 x['time_stamp'].astype(int))], axis=1).values
    dat['diff'] = dat.apply(\
                        lambda x: diff_mapper[(col, x['day'].astype(int), 
                                               x['time_stamp'].astype(int))], axis=1).values
    dat['mode'] = dat.apply(\
                        lambda x: mode_mapper[(col, x['day'].astype(int), 
                                               x['time_stamp'].astype(int))], axis=1).values
    dat['target'] = congestions[col].values
    
    trains, tests = [], []
    for period in test_periods:
        trains.append(dat[dat.index < period[0]])
        tests.append(dat[(dat.index >= period[0]) & (dat.index <= period[1])])
    return trains, tests, dat

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae

In [31]:
col = '22SE'
trains, tests, train_all = assemble(col)

In [32]:
trains[0]

,03NE-1,03SW-1,10NE-1,10SW-1,12NE-1,12SW-1,13NE-1,13SW-1,21NE-1,21NW-1,...,22SB-1,23NB-1,23SB-1,time_stamp,day,total-1,median,diff,mode,target
time,,,,,,,,,,,,,,,,,,,,,
1991-04-01 00:40:00,17.0,48.0,43.0,52.0,24.0,54.0,44.0,15.0,16.0,29.0,...,10.0,67.0,80.0,2,0,2730,40.0,0.555556,40,51.0
1991-04-01 01:00:00,46.0,15.0,43.0,52.0,24.0,54.0,44.0,15.0,16.0,29.0,...,32.0,64.0,79.0,3,0,2735,40.0,0.962963,40,51.0
1991-04-01 01:20:00,50.0,31.0,43.0,52.0,24.0,54.0,44.0,15.0,16.0,29.0,...,60.0,33.0,77.0,4,0,2692,40.0,1.000000,40,51.0
1991-04-01 01:40:00,50.0,19.0,43.0,52.0,24.0,54.0,44.0,15.0,16.0,29.0,...,43.0,50.0,79.0,5,0,2658,40.0,1.000000,40,51.0
1991-04-01 02:00:00,48.0,26.0,43.0,52.0,24.0,54.0,44.0,15.0,16.0,29.0,...,48.0,78.0,85.0,6,0,2668,40.0,0.814815,40,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-08-26 10:20:00,28.0,28.0,65.0,45.0,48.0,48.0,50.0,38.0,21.0,29.0,...,61.0,55.0,68.0,31,0,3190,20.0,0.148148,20,20.0
1991-08-26 10:40:00,24.0,23.0,72.0,43.0,49.0,60.0,40.0,39.0,21.0,29.0,...,69.0,60.0,71.0,32,0,3049,20.0,0.518519,20,20.0
1991-08-26 11:00:00,30.0,31.0,61.0,45.0,55.0,61.0,62.0,37.0,18.0,29.0,...,62.0,57.0,69.0,33,0,3090,20.0,0.592593,20,21.0


In [36]:
regrs = {}

for col in uniques:
# for col in ['22SE']:
    trains, tests, train_all = assemble(col)
    
    best_mae = 1000.0
    print(f"fitting {col}")
    for leaf in [3,5,10,20,50,100,200]:
        regr = RandomForestRegressor(n_estimators=500, min_samples_leaf=leaf, criterion='absolute_error', n_jobs=48)
        
        results = []
        for train, test in zip(trains, tests):
            X, y = train.drop(columns='target').values, train['target'].values
            regr.fit(X, y)
            
            X, y_test = test.drop(columns='target').values, test['target'].values
            y_pred = regr.predict(X)
            
            results.append(mae(y_pred, y_test))
        
        mean_mae = np.mean(results)
        print(f"{col} leaf {leaf}: {mean_mae}")
        if mean_mae < best_mae:
            best_leaf = leaf
            best_mae = mean_mae
    
    print(f"refitting with best leaf={best_leaf}")
    regr = RandomForestRegressor(n_estimators=500, min_samples_leaf=best_leaf, criterion='absolute_error', n_jobs=48)
    X, y = train_all.drop(columns='target').values, train_all['target'].values
    regr.fit(X,y)
    regrs[col] = regr

fitting 00EB
00EB leaf 3: 9.14819468208703
00EB leaf 5: 9.053955904098014
00EB leaf 10: 9.016275612589777
00EB leaf 20: 9.008127534854246
00EB leaf 50: 9.032964908111534
00EB leaf 100: 9.066252508449516
00EB leaf 200: 9.112637436628646
refitting with best leaf=20
fitting 00NB
00NB leaf 3: 7.5960426436417405
00NB leaf 5: 7.5052880756231515
00NB leaf 10: 7.447336475496409
00NB leaf 20: 7.444479377904521
00NB leaf 50: 7.371714776087875
00NB leaf 100: 7.391392427122939
00NB leaf 200: 7.361883581537811
refitting with best leaf=200
fitting 00SB
00SB leaf 3: 8.078227186311787
00SB leaf 5: 8.088135245035911
00SB leaf 10: 8.17058708280524
00SB leaf 20: 8.152028437896071
00SB leaf 50: 8.249070368610056
00SB leaf 100: 8.537873838191803
00SB leaf 200: 8.30180851288551
refitting with best leaf=3
fitting 01EB
01EB leaf 3: 4.614312711237854
01EB leaf 5: 4.555696952893959
01EB leaf 10: 4.553996224123362
01EB leaf 20: 4.612311866286438
01EB leaf 50: 4.651221878960709
01EB leaf 100: 4.708313767427123
01

fitting 12SW
12SW leaf 3: 5.470420865019012
12SW leaf 5: 5.316468446345585
12SW leaf 10: 5.15696118504436
12SW leaf 20: 5.161832646810308
12SW leaf 50: 5.207796841994085
12SW leaf 100: 5.3712235952682725
12SW leaf 200: 5.3906930713983945
refitting with best leaf=10
fitting 12WB
12WB leaf 3: 6.03557335234474
12WB leaf 5: 5.993592390156316
12WB leaf 10: 6.095150163709337
12WB leaf 20: 6.245798083016476
12WB leaf 50: 6.241960472116603
12WB leaf 100: 6.373301172370089
12WB leaf 200: 6.513441619138151
refitting with best leaf=5
fitting 13EB
13EB leaf 3: 5.451978031263202
13EB leaf 5: 5.4246246303337555
13EB leaf 10: 5.4514982836924375
13EB leaf 20: 5.4848917670046475
13EB leaf 50: 5.549182931981411
13EB leaf 100: 5.628903041825095
13EB leaf 200: 5.767624498310097
refitting with best leaf=5
fitting 13NB
13NB leaf 3: 4.635884136037178
13NB leaf 5: 4.682623811787072
13NB leaf 10: 4.707357308829742
13NB leaf 20: 4.6685220743557245
13NB leaf 50: 4.762211977186312
13NB leaf 100: 4.839249128643853

fitting 23SB
23SB leaf 3: 2.996137885509083
23SB leaf 5: 2.9254764733840304
23SB leaf 10: 2.899553680819603
23SB leaf 20: 2.9255918356569497
23SB leaf 50: 2.932889522602451
23SB leaf 100: 2.9630337716518804
23SB leaf 200: 2.9921874735952687
refitting with best leaf=10
fitting 23SW
23SW leaf 3: 4.322459204689481
23SW leaf 5: 4.279598991339248
23SW leaf 10: 4.262381125897761
23SW leaf 20: 4.261718578369244
23SW leaf 50: 4.308144909167723
23SW leaf 100: 4.332141872623574
23SW leaf 200: 4.36026299112801
refitting with best leaf=20
fitting 23WB
23WB leaf 3: 8.537775110899872
23WB leaf 5: 8.508476552598225
23WB leaf 10: 8.52540589353612
23WB leaf 20: 8.552513968103085
23WB leaf 50: 8.51770262991128
23WB leaf 100: 8.578320870299958
23WB leaf 200: 8.609640869243767
refitting with best leaf=5


In [37]:
import pickle

In [38]:
with open('baseline2_models.pickle', 'wb') as fid:
    pickle.dump(regrs, fid)

In [39]:
regrs = {}

for col in uniques:
# for col in ['22SE']:
    trains, tests, train_all = assemble(col)
    
    best_mae = 1000.0
    print(f"fitting {col}")
    for leaf in [3,5,10,20,50,100,200]:
        regr = RandomForestRegressor(n_estimators=100, min_samples_leaf=leaf, criterion='absolute_error', n_jobs=48)
        
        results = []
        for train, test in zip(trains, tests):
            X, y = train.drop(columns='target').values, train['target'].values
            regr.fit(X, y)
            
            X, y_test = test.drop(columns='target').values, test['target'].values
            y_pred = regr.predict(X)
            
            results.append(mae(y_pred, y_test))
        
        mean_mae = np.mean(results)
        print(f"{col} leaf {leaf}: {mean_mae}")
        if mean_mae < best_mae:
            best_leaf = leaf
            best_mae = mean_mae
    
    print(f"refitting with best leaf={best_leaf}")
    regr = RandomForestRegressor(n_estimators=100, min_samples_leaf=best_leaf, criterion='absolute_error', n_jobs=48)
    X, y = train_all.drop(columns='target').values, train_all['target'].values
    regr.fit(X,y)
    regrs[col] = regr

fitting 00EB
00EB leaf 3: 9.103190747782003
00EB leaf 5: 9.046383475918883
00EB leaf 10: 9.002254171947612
00EB leaf 20: 8.943317622517956
00EB leaf 50: 8.975769565906209
00EB leaf 100: 9.076062658428391
00EB leaf 200: 9.135429340937897
refitting with best leaf=20
fitting 00NB
00NB leaf 3: 7.621776906421632
00NB leaf 5: 7.505741576890579
00NB leaf 10: 7.486982863329109
00NB leaf 20: 7.414265684410646
00NB leaf 50: 7.384384373679764
00NB leaf 100: 7.390221403675538
00NB leaf 200: 7.348819708491762
refitting with best leaf=200
fitting 00SB
00SB leaf 3: 8.167432799957751
00SB leaf 5: 8.218132129277565
00SB leaf 10: 8.163326732150402
00SB leaf 20: 8.158131733206591
00SB leaf 50: 8.20291429024081
00SB leaf 100: 8.538334389522603
00SB leaf 200: 8.296801594845796
refitting with best leaf=20
fitting 01EB
01EB leaf 3: 4.638475654837347
01EB leaf 5: 4.527807483100972
01EB leaf 10: 4.567232784114912
01EB leaf 20: 4.575209785593579
01EB leaf 50: 4.675352371144909
01EB leaf 100: 4.727106437473595
0

fitting 12SW
12SW leaf 3: 5.453535593578369
12SW leaf 5: 5.305434489860583
12SW leaf 10: 5.19708557773553
12SW leaf 20: 5.137657900295733
12SW leaf 50: 5.229422000422476
12SW leaf 100: 5.3506741128010145
12SW leaf 200: 5.395984500422475
refitting with best leaf=20
fitting 12WB
12WB leaf 3: 6.114055502746091
12WB leaf 5: 6.010740520701309
12WB leaf 10: 6.13246805027461
12WB leaf 20: 6.281135007393325
12WB leaf 50: 6.26171393113646
12WB leaf 100: 6.367709653569919
12WB leaf 200: 6.489574223700886
refitting with best leaf=5
fitting 13EB
13EB leaf 3: 5.502924324038868
13EB leaf 5: 5.432978057667934
13EB leaf 10: 5.464131812420785
13EB leaf 20: 5.543176621250528
13EB leaf 50: 5.518309305027461
13EB leaf 100: 5.626734526827207
13EB leaf 200: 5.778750396070976
refitting with best leaf=5
fitting 13NB
13NB leaf 3: 4.628203158005914
13NB leaf 5: 4.721261882129278
13NB leaf 10: 4.708068757921419
13NB leaf 20: 4.67880729826785
13NB leaf 50: 4.748551568441064
13NB leaf 100: 4.848005650612589
13NB l

fitting 23SB
23SB leaf 3: 2.963337426066752
23SB leaf 5: 2.910200147866498
23SB leaf 10: 2.9019777144064216
23SB leaf 20: 2.915729694761302
23SB leaf 50: 2.930875976975074
23SB leaf 100: 2.9623481727925642
23SB leaf 200: 2.977098780101394
refitting with best leaf=10
fitting 23SW
23SW leaf 3: 4.261469291297001
23SW leaf 5: 4.251892295099282
23SW leaf 10: 4.284908639628221
23SW leaf 20: 4.254393615335868
23SW leaf 50: 4.312503300591466
23SW leaf 100: 4.320126082594001
23SW leaf 200: 4.369681162864385
refitting with best leaf=5
fitting 23WB
23WB leaf 3: 8.593879251161809
23WB leaf 5: 8.561627851711027
23WB leaf 10: 8.576085234474018
23WB leaf 20: 8.590620379171948
23WB leaf 50: 8.491440774186733
23WB leaf 100: 8.582277144064218
23WB leaf 200: 8.606693995564004
refitting with best leaf=50


In [40]:
with open('baseline2_models_forcompare.pickle', 'wb') as fid:
    pickle.dump(regrs, fid)

In [41]:
with open('baseline2_models.pickle','rb') as fid:
    regrs = pickle.load(fid)

In [42]:
test = pd.read_csv('test.csv')
test.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


In [48]:
def preprocess_test(dat, time_mapper):
    dat['unique'] = dat['x'].astype(str) + dat['y'].astype(str) + dat['direction']
    dat['day'] = pd.to_datetime(dat['time']).dt.weekday
    dat['time_stamp'] = dat['time'].apply(lambda x:time_mapper[x.split()[1][:5]])
    congestions = dat[['unique','time']].set_index('time')
    congestions['congestion'] = 0.0
    congestions = congestions.groupby(['unique','time']).mean().unstack('unique')
    congestions.columns = congestions.columns.droplevel(0)
    
    for col in congestions.columns.to_list():
        col_new = col + '-1'
        congestions[col_new] = 0.0
    congestions = congestions.copy()
    
    congestions['time_stamp'] = \
       pd.Series(congestions.index).apply(lambda x:time_mapper[x.split()[1][:5]]).values
    congestions['day'] = \
       pd.to_datetime(pd.Series(congestions.index)).dt.weekday.values
    congestions['total-1'] = 0.0
    
    return congestions

In [49]:
congestions_test = preprocess_test(test, time_mapper)
congestions_test

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,22WB-1,23EB-1,23NB-1,23NE-1,23SB-1,23SW-1,23WB-1,time_stamp,day,total-1
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,0,0.0
1991-09-30 12:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,0,0.0
1991-09-30 12:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,0,0.0
1991-09-30 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,0,0.0
1991-09-30 13:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,0,0.0
1991-09-30 13:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41,0,0.0
1991-09-30 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42,0,0.0
1991-09-30 14:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,0,0.0
1991-09-30 14:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44,0,0.0


In [52]:
congestions.tail(1)

,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day,total,total-1,total-2
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 11:40:00,70.0,40.0,51.0,24.0,65.0,52.0,42.0,43.0,62.0,77.0,...,69.0,22.0,19.0,53.0,37.0,35,0,3166,3079,2952


In [50]:
col_mapper = {}
cols_tosum = []
for col in uniques:
    col_mapper[col+'-1'] = col
    cols_tosum.append(col+'-1')

In [53]:
idx_test = congestions_test.index[0]
idx_train = congestions.index[-1]
for col in uniques:
    col1 = col+'-1'
    congestions_test.at[idx_test, col1] = congestions.at[idx_train, col_mapper[col1]]
congestions_test.at[idx_test, 'total-1'] = congestions_test.loc[idx_test, cols_tosum].sum()
congestions_test

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,22WB-1,23EB-1,23NB-1,23NE-1,23SB-1,23SW-1,23WB-1,time_stamp,day,total-1
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,71.0,48.0,54.0,28.0,68.0,17.0,24.0,36,0,3166.0
1991-09-30 12:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,0,0.0
1991-09-30 12:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,0,0.0
1991-09-30 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,0,0.0
1991-09-30 13:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,0,0.0
1991-09-30 13:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41,0,0.0
1991-09-30 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42,0,0.0
1991-09-30 14:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,0,0.0
1991-09-30 14:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44,0,0.0


In [56]:
train_all.tail(1)

,03EB-1,03WB-1,13EB-1,13WB-1,23EB-1,23WB-1,time_stamp,day,total-1,median,diff,mode,target
time,,,,,,,,,,,,,
1991-09-30 11:40:00,25.0,56.0,38.0,37.0,57.0,53.0,35,0,3079,37.0,0.074074,37,24.0


In [70]:
connected['00EB']

['00EB-1', '10EB-1', '10WB-1', '20EB-1', '20WB-1']

In [73]:
def predict():
    
    for i in range(len(congestions_test.index)):
        
        # fill the new row
        idx = congestions_test.index[i]
        if (i>0):
            idx1 = congestions_test.index[i-1]
            for col in uniques:
                congestions_test.at[idx, col+'-1'] = congestions_test.at[idx1, col]
            congestions_test.at[idx, 'total-1'] = congestions_test.loc[idx, cols_tosum].sum()
        
        # make prediction of this row
        for col in uniques:
            cols = connected[col][:]
            cols.append('time_stamp')
            cols.append('day')
            cols.append('total-1')
            dat = congestions_test.loc[[idx], cols]
            dat['median'] = dat.apply(\
                        lambda x: median_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            dat['diff'] = dat.apply(\
                        lambda x: diff_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            dat['mode'] = dat.apply(\
                        lambda x: mode_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            y = regrs[col].predict(dat.values)
            congestions_test.at[idx, col] = y

In [74]:
predict()

In [75]:
congestions_test.head()

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,22WB-1,23EB-1,23NB-1,23NE-1,23SB-1,23SW-1,23WB-1,time_stamp,day,total-1
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 12:00:00,46.874,36.090,55.476,22.300,69.456,59.823,45.315,47.438,59.897,74.181,...,71.000,48.000,54.000,28.000,68.000,17.000,24.000,36,0,3166.000
1991-09-30 12:20:00,44.649,35.710,51.925,26.500,71.460,62.929,42.944,49.431,58.130,68.235,...,73.764,47.460,58.576,27.100,69.856,18.752,44.075,37,0,3234.295
1991-09-30 12:40:00,46.975,37.034,53.991,27.124,71.583,62.623,40.532,48.346,58.422,65.319,...,73.319,45.354,56.853,27.390,69.963,20.294,41.174,38,0,3248.693
1991-09-30 13:00:00,48.414,37.307,51.685,26.647,72.283,63.535,45.354,46.949,61.454,63.757,...,74.380,49.761,58.970,25.980,70.701,15.471,49.475,39,0,3269.544
1991-09-30 13:20:00,45.270,36.327,49.217,29.698,73.289,62.613,45.355,49.295,61.768,64.432,...,73.294,45.016,59.478,25.274,70.775,18.076,46.168,40,0,3286.831


In [76]:
congestions.tail(1)

,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day,total,total-1,total-2
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 11:40:00,70.0,40.0,51.0,24.0,65.0,52.0,42.0,43.0,62.0,77.0,...,69.0,22.0,19.0,53.0,37.0,35,0,3166,3079,2952


In [78]:
congestions_test[['00EB','00EB-1']]

unique,00EB,00EB-1
time,,
1991-09-30 12:00:00,46.874,70.000
1991-09-30 12:20:00,44.649,46.874
1991-09-30 12:40:00,46.975,44.649
1991-09-30 13:00:00,48.414,46.975
1991-09-30 13:20:00,45.270,48.414
1991-09-30 13:40:00,56.364,45.270
1991-09-30 14:00:00,52.047,56.364
1991-09-30 14:20:00,51.170,52.047
1991-09-30 14:40:00,55.333,51.170


In [79]:
test['congestion'] = 0.0
for idx in test.index:
    time, unique = test.at[idx,'time'], test.at[idx,'unique']
    test.at[idx,'congestion'] = congestions_test.at[time,unique]
test.head()

,row_id,time,x,y,direction,unique,day,time_stamp,congestion
0,848835,1991-09-30 12:00:00,0,0,EB,00EB,0,36,46.874
1,848836,1991-09-30 12:00:00,0,0,NB,00NB,0,36,36.090
2,848837,1991-09-30 12:00:00,0,0,SB,00SB,0,36,55.476
3,848838,1991-09-30 12:00:00,0,1,EB,01EB,0,36,22.300
4,848839,1991-09-30 12:00:00,0,1,NB,01NB,0,36,69.456


In [80]:
test[['row_id', 'congestion']].to_csv('submission_baseline2.csv', index=False)

In [ ]:
dat.head()

In [ ]:
lower = dat.groupby(['unique','day','time_stamp'])['congestion'].quantile(0.15).to_dict()
upper = dat.groupby(['unique','day','time_stamp'])['congestion'].quantile(0.7).to_dict()

In [ ]:
lower

In [ ]:
test

In [ ]:
test['lower'] = test.apply(lambda x:\
    lower[(x['unique'], x['day'], x['time_stamp'])], axis=1)
test['upper'] = test.apply(lambda x:\
    upper[(x['unique'], x['day'], x['time_stamp'])], axis=1)
test

In [ ]:
test['congestion_tweak'] = test['congestion'].clip(test['lower'],test['upper'])

In [ ]:
test[['row_id', 'congestion_tweak']].to_csv('submission_baseline1_tweaked.csv', index=False)